# **Stock Project**

# **Imports**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests
import json
import warnings
warnings.filterwarnings("ignore")
import pickle
import datetime as dt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveRegressor
from bs4 import BeautifulSoup

# **Obtain Price Function**

In [2]:
# Create Function
def getData (symbol):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    url = f'https://finance.yahoo.com/quote/{symbol}'
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    stock ={
    'symbol': symbol,    
    'price' : soup.find('div',{'class':'D(ib) Va(m) Maw(65%) Ov(h)'}).find_all('fin-streamer')[0].text,
    'Pchange' : soup.find('div',{'class':'D(ib) Va(m) Maw(65%) Ov(h)'}).find_all('fin-streamer')[1].text,
    'RMchange' : soup.find('div',{'class':'D(ib) Va(m) Maw(65%) Ov(h)'}).find_all('span')[1].text,
    }
    return stock

In [3]:
mystocks = ['JNJ?p=JNJ', 'AI?p=AI', 'GOLD?p=GOLD', 'SECO?p=SECO', 'CAT?p=CAT']

In [4]:
stockdata = []

In [5]:
# Create a Loop 
for item in mystocks:
    stockdata.append(getData(item))
    print('Getting: ',item)
with open('stockdata.json', 'w') as f:
    json.dump(stockdata, f)
    print(stockdata)
print('Fin.')

Getting:  JNJ?p=JNJ
Getting:  AI?p=AI
Getting:  GOLD?p=GOLD
Getting:  SECO?p=SECO
Getting:  CAT?p=CAT
[{'symbol': 'JNJ?p=JNJ', 'price': '165.15', 'Pchange': '-0.46', 'RMchange': '(-0.28%)'}, {'symbol': 'AI?p=AI', 'price': '22.84', 'Pchange': '+1.75', 'RMchange': '(+8.30%)'}, {'symbol': 'GOLD?p=GOLD', 'price': '19.73', 'Pchange': '+0.05', 'RMchange': '(+0.25%)'}, {'symbol': 'SECO?p=SECO', 'price': '1.2100', 'Pchange': '+0.3295', 'RMchange': '(+37.42%)'}, {'symbol': 'CAT?p=CAT', 'price': '209.17', 'Pchange': '-4.36', 'RMchange': '(-2.04%)'}]
Fin.
